In [1]:
import os
import pandas as pd
import requests
from ftplib import FTP 


In [237]:
folder='all'
columns=['handle','url','template-type','maintainer-email','name']
all_items=[]
for file in os.listdir(folder):
    if file.endswith('arch.rdf'):
        with open(folder+'/'+file,'r') as rdf_file:
            rdf_string=rdf_file.read()
            rdf_list= rdf_string.replace('\xef\xbb\xbf','').split('\n')
            rdf_item_dict={}
            for rdf_item in rdf_list:
                if ':' not in rdf_item: 
                    continue
                key=rdf_item.split(':')[0]
                if key.lower() in columns:
                    value=rdf_item.replace(key+':','')
                    rdf_item_dict[key.strip().lower()]=value.strip()
            all_items.append(rdf_item_dict)

In [234]:
df=pd.DataFrame(all_items)
df.shape

(1903, 5)

In [236]:
all_items_status=[]
for item in all_items:
    if 'template-type' in item:
        if item['template-type']=='ReDIF-Archive 1.0' and item['url'][:4]=='http':
            url=item['url']
            try: 
                r = requests.get(url)
                if r.status_code==200: item['status']='OK'
                if r.status_code==404: item['status']='Page Not Found'
            except:
                item['status'] = 'Connectoin Refused'
            all_items_status.append(item)

In [238]:
df_http=pd.DataFrame(all_items_status)
df_http.shape

(1626, 6)

In [239]:
float(df_http[df_http.status!='OK'].shape[0])/df_http.shape[0]

0.14821648216482164

In [240]:
all_items_status_ftp=[]
for item in all_items:
    if 'template-type' in item:
        if item['template-type']=='ReDIF-Archive 1.0' and item['url'][:3]=='ftp':
            url=item['url'].split('/')[2]
            try: 
                ftp = FTP(url)
                item['status']='OK'
            except:
                item['status'] = 'Connectoin Refused'
            all_items_status_ftp.append(item)

In [241]:
df_ftp=pd.DataFrame(all_items_status_ftp)
df_ftp.shape

(256, 6)

In [242]:
float(df_ftp[df_ftp.status!='OK'].shape[0])/df_ftp.shape[0]

0.09765625

In [243]:
df_all=df_http.append(df_ftp)
df_all.shape

(1882, 6)

In [244]:
df_all['handle']=df_all['handle'].str.lower().str[6:9]
df_all.shape

(1882, 6)

In [245]:
df_all.sort_values('handle')[['handle','name','url','status']].to_csv('archives.csv',index=False)

In [246]:
for item in all_items:
    if 'handle' in item and item['handle']=='RePEc:adb':
        print item

{'url': 'https://www.afdb.org/RePEc/adb/', 'template-type': 'ReDIF-Archive 1.0', 'handle': 'RePEc:adb', 'name': 'African Development Bank', 'maintainer-email': 's.adjatan@afdb.org'}


In [247]:
len(all_items)

1903

In [249]:
df_all[df_all['handle']=='RePEc:adb']

,handle,maintainer-email,name,status,template-type,url
